<a href="https://colab.research.google.com/github/hfwalvir/Dissertation/blob/main/ProactiveSecretSharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random



In [ ]:
# File path - adjust this to match your system
file_path = '/content/sample_data/combined_financial_data_idx.csv'
# Read the CSV file
data = pd.read_csv(file_path)
data.head()

,symbol,account,type,2020,2021,2022,2023
0,AALI,Accounts Payable,BS,7.702640e+11,1.026717e+12,1.224423e+12,8.420640e+11
1,AALI,Accounts Receivable,BS,7.658490e+11,4.581350e+11,8.487700e+11,6.744870e+11
2,AALI,Accumulated Depreciation,BS,-1.092095e+13,-1.213381e+13,-1.330375e+13,-1.443685e+13
3,AALI,Additional Paid In Capital,BS,3.878995e+12,3.878995e+12,3.878995e+12,3.878995e+12
4,AALI,Allowance For Doubtful Accounts Receivable,BS,-2.426100e+10,-2.454300e+10,-2.705700e+10,-2.651600e+10


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89243 entries, 0 to 89242
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   symbol   89243 non-null  object 
 1   account  89243 non-null  object 
 2   type     89243 non-null  object 
 3   2020     83988 non-null  float64
 4   2021     84474 non-null  float64
 5   2022     83634 non-null  float64
 6   2023     69831 non-null  float64
dtypes: float64(4), object(3)
memory usage: 4.8+ MB


# **ACTUAL RATIOS**

In [ ]:
# Filter data for AALI and the year 2023
aali_2023 = data[(data['symbol'] == 'AALI') & (data['2023'].notna())]

# Extract relevant financial data
financial_data = {
    'current_assets': float(aali_2023[aali_2023['account'] == 'Current Assets']['2023'].values[0]),
    'current_liabilities': float(aali_2023[aali_2023['account'] == 'Current Liabilities']['2023'].values[0]),
    'total_liabilities': float(aali_2023[aali_2023['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0]),
    'total_equity': float(aali_2023[aali_2023['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]),
    'total_assets': float(aali_2023[aali_2023['account'] == 'Total Assets']['2023'].values[0]),
    'net_income': float(aali_2023[aali_2023['account'] == 'Net Income']['2023'].values[0]),
    'operating_revenue': float(aali_2023[aali_2023['account'] == 'Operating Revenue']['2023'].values[0])
}

# Calculate ratios
current_ratio = financial_data['current_assets'] / financial_data['current_liabilities']
debt_to_equity = financial_data['total_liabilities'] / financial_data['total_equity']
roa = financial_data['net_income'] / financial_data['total_assets']
profit_margin = financial_data['net_income'] / financial_data['operating_revenue']

# Print results
print(f"Current Ratio: {current_ratio:.2f}")
print(f"Debt-to-Equity Ratio: {debt_to_equity:.2f}")
print(f"Return on Assets (ROA): {roa:.2%}")
print(f"Profit Margin: {profit_margin:.2%}")


Current Ratio: 1.83
Debt-to-Equity Ratio: 0.28
Return on Assets (ROA): 3.66%
Profit Margin: 5.09%


# **ACCOUNTING EQUATION CALCULATION (VERIFICATION)**

In [ ]:
# Filter for AALI and balance sheet items
aali_bs = data[(data['symbol'] == 'AALI') & (data['type'] == 'BS')]

# Extracting relevant balance sheet items for 2023
total_assets = aali_bs[aali_bs['account'] == 'Total Assets']['2023'].values[0]
total_liabilities = aali_bs[aali_bs['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0]
total_equity = aali_bs[aali_bs['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]

# Converting scientific notation to integers
total_assets = int(float(total_assets))
total_liabilities = int(float(total_liabilities))
total_equity = int(float(total_equity))

print(f"TOTAL ASSETS: {total_assets}")
print(f"TOTAL LIABILITIES: {total_liabilities}")
print(f"TOTAL EQUITY: {total_equity}")

# Function to generate shares
def generate_shares(secret, num_shares):
    shares = [random.randint(0, secret) for _ in range(num_shares - 1)]
    shares.append(secret - sum(shares))
    return shares

# Function to reconstruct secret
def reconstruct_secret(shares):
    return sum(shares)

# Function to refresh shares
def refresh_shares(shares):
    new_shares = generate_shares(0, len(shares))
    return [s + n for s, n in zip(shares, new_shares)]

# Generate initial shares
assets_shares = generate_shares(total_assets, 3)
liabilities_shares = generate_shares(total_liabilities, 3)
equity_shares = generate_shares(total_equity, 3)

print("\nInitial Shares:")
print(f"Assets: {assets_shares}")
print(f"Liabilities: {liabilities_shares}")
print(f"Equity: {equity_shares}")

# Verify balance sheet equation
reconstructed_assets = reconstruct_secret(assets_shares)
reconstructed_liabilities = reconstruct_secret(liabilities_shares)
reconstructed_equity = reconstruct_secret(equity_shares)

print("\nReconstructed Figures:")
print(f"Assets: {reconstructed_assets}")
print(f"Liabilities + Equity: {reconstructed_liabilities + reconstructed_equity}")
print(f"Difference: {reconstructed_assets - (reconstructed_liabilities + reconstructed_equity)}")

# Proactive refresh
for i in range(3):
    print(f"\nRefresh {i+1}:")
    assets_shares = refresh_shares(assets_shares)
    liabilities_shares = refresh_shares(liabilities_shares)
    equity_shares = refresh_shares(equity_shares)

    print(f"Assets: {assets_shares}")
    print(f"Liabilities: {liabilities_shares}")
    print(f"Equity: {equity_shares}")

    # Verify after refresh
    reconstructed_assets = reconstruct_secret(assets_shares)
    reconstructed_liabilities = reconstruct_secret(liabilities_shares)
    reconstructed_equity = reconstruct_secret(equity_shares)

    print(f"Assets: {reconstructed_assets}")
    print(f"Liabilities + Equity: {reconstructed_liabilities + reconstructed_equity}")
    print(f"Difference: {reconstructed_assets - (reconstructed_liabilities + reconstructed_equity)}")

TOTAL ASSETS: 28846243000000
TOTAL LIABILITIES: 6280237000000
TOTAL EQUITY: 22566006000000

Initial Shares:
Assets: [24793107389377, 1707296930192, 2345838680431]
Liabilities: [4848406747797, 1424594206751, 7236045452]
Equity: [3561616082430, 5806580709381, 13197809208189]

Reconstructed Figures:
Assets: 28846243000000
Liabilities + Equity: 28846243000000
Difference: 0

Refresh 1:
Assets: [24793107389377, 1707296930192, 2345838680431]
Liabilities: [4848406747797, 1424594206751, 7236045452]
Equity: [3561616082430, 5806580709381, 13197809208189]
Assets: 28846243000000
Liabilities + Equity: 28846243000000
Difference: 0

Refresh 2:
Assets: [24793107389377, 1707296930192, 2345838680431]
Liabilities: [4848406747797, 1424594206751, 7236045452]
Equity: [3561616082430, 5806580709381, 13197809208189]
Assets: 28846243000000
Liabilities + Equity: 28846243000000
Difference: 0

Refresh 3:
Assets: [24793107389377, 1707296930192, 2345838680431]
Liabilities: [4848406747797, 1424594206751, 7236045452]
E

# **RATIO ANALYSIS WITH ONE COMPANY - AALI**

In [ ]:
import pandas as pd
import random
#Defining functions for share construction , reconstruction as well as division for ratio calculations
def generate_shares(secret, num_shares, prime):
    shares = [random.randint(0, prime-1) for _ in range(num_shares - 1)]
    shares.append((secret - sum(shares)) % prime)
    return shares

def reconstruct_secret(shares, prime):
    return sum(shares) % prime

def refresh_shares(shares, prime):
    new_shares = generate_shares(0, len(shares), prime)
    return [(s + n) % prime for s, n in zip(shares, new_shares)]

def secure_division(numerator, denominator, prime):
    return (numerator * pow(denominator, prime-2, prime)) % prime

# Prime number for modular arithmetic
prime = 1000000007


# Filter data for AALI and the year 2023
aali_2023 = data[(data['symbol'] == 'AALI') & (data['2023'].notna())]

# Extract relevant financial data
financial_data = {
    'current_assets': int(float(aali_2023[aali_2023['account'] == 'Current Assets']['2023'].values[0])),
    'current_liabilities': int(float(aali_2023[aali_2023['account'] == 'Current Liabilities']['2023'].values[0])),
    'total_liabilities': int(float(aali_2023[aali_2023['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0])),
    'total_equity': int(float(aali_2023[aali_2023['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0])),
    'total_assets': int(float(aali_2023[aali_2023['account'] == 'Total Assets']['2023'].values[0])),
    'net_income': int(float(aali_2023[aali_2023['account'] == 'Net Income']['2023'].values[0])),
    'total_revenue': int(float(aali_2023[aali_2023['account'] == 'Total Revenue']['2023'].values[0]))
}

# Number of shares (participants)
num_shares = 3

# Generate shares for financial data
shared_data = {key: generate_shares(value, num_shares, prime) for key, value in financial_data.items()}

def calculate_ratios(shared_data):
    current_ratio = secure_division(
        reconstruct_secret(shared_data['current_assets'], prime),
        reconstruct_secret(shared_data['current_liabilities'], prime),
        prime
    )
    debt_to_equity = secure_division(
        reconstruct_secret(shared_data['total_liabilities'], prime),
        reconstruct_secret(shared_data['total_equity'], prime),
        prime
    )
    roa = secure_division(
        reconstruct_secret(shared_data['net_income'], prime),
        reconstruct_secret(shared_data['total_assets'], prime),
        prime
    )
    profit_margin = secure_division(
        reconstruct_secret(shared_data['net_income'], prime),
        reconstruct_secret(shared_data['total_revenue'], prime),
        prime
    )
    return current_ratio, debt_to_equity, roa, profit_margin

print("Initial Ratios:")
initial_ratios = calculate_ratios(shared_data)
print(f"Current Ratio: {initial_ratios[0]}")
print(f"Debt-to-Equity Ratio: {initial_ratios[1]}")
print(f"Return on Assets: {initial_ratios[2]}")
print(f"Profit Margin: {initial_ratios[3]}")

# Proactive refresh and recalculation
for i in range(3):
    print(f"\nRefresh {i+1}:")
    for key in shared_data:
        shared_data[key] = refresh_shares(shared_data[key], prime)

    refreshed_ratios = calculate_ratios(shared_data)
    print(f"Current Ratio: {refreshed_ratios[0]}")
    print(f"Debt-to-Equity Ratio: {refreshed_ratios[1]}")
    print(f"Return on Assets: {refreshed_ratios[2]}")
    print(f"Profit Margin: {refreshed_ratios[3]}")
def convert_to_real_ratio(secure_ratio, prime):
    return secure_ratio / prime

# After ca=onverting them to ratios by dividing by the prime modular arithmetic
real_current_ratio = convert_to_real_ratio(refreshed_ratios[0], prime)
real_debt_to_equity = convert_to_real_ratio(refreshed_ratios[1], prime)
real_roa = convert_to_real_ratio(refreshed_ratios[2], prime)
real_profit_margin = convert_to_real_ratio(refreshed_ratios[3], prime)

print(f"\nReal Current Ratio: {real_current_ratio:.2f}")
print(f"Real Debt-to-Equity Ratio: {real_debt_to_equity:.2f}")
print(f"Real Return on Assets: {real_roa:.2f}")
print(f"Real Profit Margin: {real_profit_margin:.2f}")

Initial Ratios:
Current Ratio: 873546332
Debt-to-Equity Ratio: 248179188
Return on Assets: 906832761
Profit Margin: 61947636

Refresh 1:
Current Ratio: 873546332
Debt-to-Equity Ratio: 248179188
Return on Assets: 906832761
Profit Margin: 61947636

Refresh 2:
Current Ratio: 873546332
Debt-to-Equity Ratio: 248179188
Return on Assets: 906832761
Profit Margin: 61947636

Refresh 3:
Current Ratio: 873546332
Debt-to-Equity Ratio: 248179188
Return on Assets: 906832761
Profit Margin: 61947636

Real Current Ratio: 0.87
Real Debt-to-Equity Ratio: 0.25
Real Return on Assets: 0.91
Real Profit Margin: 0.06


# **TRYING PROACTIVE SECRET SHARING WITH NEW COMPANY ADDITION : ABDA**

In [ ]:
import pandas as pd
import random

# Function to generate shares
def generate_shares(secret, num_shares):
    shares = [random.randint(0, secret) for _ in range(num_shares - 1)]
    shares.append(secret - sum(shares))
    return shares

# Function to reconstruct secret
def reconstruct_secret(shares):
    return sum(shares)

# Function to refresh shares
def refresh_shares(shares):
    new_shares = generate_shares(0, len(shares))
    return [s + n for s, n in zip(shares, new_shares)]

# Function to extract balance sheet data for a company
def extract_bs_data(data, symbol):
    company_bs = data[(data['symbol'] == symbol) & (data['type'] == 'BS')]
    if company_bs.empty:
        return None, None, None
    total_assets = int(float(company_bs[company_bs['account'] == 'Total Assets']['2023'].values[0]))
    total_liabilities = int(float(company_bs[company_bs['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0]))
    total_equity = int(float(company_bs[company_bs['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]))
    return total_assets, total_liabilities, total_equity

# Function to perform proactive secret sharing for a company
def proactive_secret_sharing(company_name, assets, liabilities, equity):
    print(f"\n{company_name} Balance Sheet:")
    print(f"TOTAL ASSETS: {assets}")
    print(f"TOTAL LIABILITIES: {liabilities}")
    print(f"TOTAL EQUITY: {equity}")

    # Generate initial shares
    num_shares = 3
    assets_shares = generate_shares(assets, num_shares)
    liabilities_shares = generate_shares(liabilities, num_shares)
    equity_shares = generate_shares(equity, num_shares)

    print("\nInitial Shares:")
    print(f"Assets: {assets_shares}")
    print(f"Liabilities: {liabilities_shares}")
    print(f"Equity: {equity_shares}")

    # Verify balance sheet equation
    reconstructed_assets = reconstruct_secret(assets_shares)
    reconstructed_liabilities = reconstruct_secret(liabilities_shares)
    reconstructed_equity = reconstruct_secret(equity_shares)

    print("\nReconstructed Figures:")
    print(f"Assets: {reconstructed_assets}")
    print(f"Liabilities + Equity: {reconstructed_liabilities + reconstructed_equity}")
    print(f"Difference: {reconstructed_assets - (reconstructed_liabilities + reconstructed_equity)}")

    # Proactive refresh
    for i in range(3):
        print(f"\nRefresh {i+1}:")
        assets_shares = refresh_shares(assets_shares)
        liabilities_shares = refresh_shares(liabilities_shares)
        equity_shares = refresh_shares(equity_shares)

        print(f"Assets: {assets_shares}")
        print(f"Liabilities: {liabilities_shares}")
        print(f"Equity: {equity_shares}")

        # Verify after refresh
        reconstructed_assets = reconstruct_secret(assets_shares)
        reconstructed_liabilities = reconstruct_secret(liabilities_shares)
        reconstructed_equity = reconstruct_secret(equity_shares)

        print(f"Assets: {reconstructed_assets}")
        print(f"Liabilities + Equity: {reconstructed_liabilities + reconstructed_equity}")
        print(f"Difference: {reconstructed_assets - (reconstructed_liabilities + reconstructed_equity)}")

# Read the CSV file
data = pd.read_csv('/content/sample_data/combined_financial_data_idx.csv')

# Extract data for AALI and ABDA
aali_assets, aali_liabilities, aali_equity = extract_bs_data(data, 'AALI')
abda_assets, abda_liabilities, abda_equity = extract_bs_data(data, 'ABDA')

# Perform proactive secret sharing for AALI and ABDA
if all(v is not None for v in [aali_assets, aali_liabilities, aali_equity]):
    proactive_secret_sharing("AALI", aali_assets, aali_liabilities, aali_equity)
else:
    print("Invalid or missing data for AALI")

if all(v is not None for v in [abda_assets, abda_liabilities, abda_equity]):
    proactive_secret_sharing("ABDA", abda_assets, abda_liabilities, abda_equity)
else:
    print("Invalid or missing data for ABDA")


AALI Balance Sheet:
TOTAL ASSETS: 28846243000000
TOTAL LIABILITIES: 6280237000000
TOTAL EQUITY: 22566006000000

Initial Shares:
Assets: [4508319955388, 22122436747051, 2215486297561]
Liabilities: [1331548486962, 4608637524878, 340050988160]
Equity: [11637138362865, 644469813271, 10284397823864]

Reconstructed Figures:
Assets: 28846243000000
Liabilities + Equity: 28846243000000
Difference: 0

Refresh 1:
Assets: [4508319955388, 22122436747051, 2215486297561]
Liabilities: [1331548486962, 4608637524878, 340050988160]
Equity: [11637138362865, 644469813271, 10284397823864]
Assets: 28846243000000
Liabilities + Equity: 28846243000000
Difference: 0

Refresh 2:
Assets: [4508319955388, 22122436747051, 2215486297561]
Liabilities: [1331548486962, 4608637524878, 340050988160]
Equity: [11637138362865, 644469813271, 10284397823864]
Assets: 28846243000000
Liabilities + Equity: 28846243000000
Difference: 0

Refresh 3:
Assets: [4508319955388, 22122436747051, 2215486297561]
Liabilities: [1331548486962, 4

# **TRYING PROACTIVE SECRET SHARING WITH 4 COMPANIES : AALI, ABMM,ACES,ACST**

# **ACTUAL RATIOS**

In [ ]:
import pandas as pd
# List of companies
companies = ['AALI', 'ABMM', 'ACES', 'ACST']

# Function to extract financial data for a company
def extract_financial_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    return {
        'current_assets': float(company_data[company_data['account'] == 'Current Assets']['2023'].values[0]),
        'current_liabilities': float(company_data[company_data['account'] == 'Current Liabilities']['2023'].values[0]),
        'total_liabilities': float(company_data[company_data['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0]),
        'total_equity': float(company_data[company_data['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]),
        'total_assets': float(company_data[company_data['account'] == 'Total Assets']['2023'].values[0]),
        'net_income': float(company_data[company_data['account'] == 'Net Income']['2023'].values[0]),
        'operating_revenue': float(company_data[company_data['account'] == 'Operating Revenue']['2023'].values[0])
    }

# Extract data for all companies
company_data = {company: extract_financial_data(company) for company in companies}

# Function to calculate ratios
def calculate_ratios(data):
    current_ratio = data['current_assets'] / data['current_liabilities']
    debt_to_equity = data['total_liabilities'] / data['total_equity']
    roa = data['net_income'] / data['total_assets']
    profit_margin = data['net_income'] / data['operating_revenue']
    return current_ratio, debt_to_equity, roa, profit_margin

# Calculate and print ratios for each company
for company in companies:
    ratios = calculate_ratios(company_data[company])
    print(f"\n{company} Ratios:")
    print(f"Current Ratio: {ratios[0]:.2f}")
    print(f"Debt-to-Equity Ratio: {ratios[1]:.2f}")
    print(f"Return on Assets (ROA): {ratios[2]:.2%}")
    print(f"Profit Margin: {ratios[3]:.2%}")


AALI Ratios:
Current Ratio: 1.83
Debt-to-Equity Ratio: 0.28
Return on Assets (ROA): 3.66%
Profit Margin: 5.09%

ABMM Ratios:
Current Ratio: 0.99
Debt-to-Equity Ratio: 1.84
Return on Assets (ROA): 13.40%
Profit Margin: 19.36%

ACES Ratios:
Current Ratio: 7.42
Debt-to-Equity Ratio: 0.25
Return on Assets (ROA): 9.85%
Profit Margin: 10.03%

ACST Ratios:
Current Ratio: 1.00
Debt-to-Equity Ratio: 5.59
Return on Assets (ROA): -10.36%
Profit Margin: -11.50%


In [ ]:
import pandas as pd
import random

def generate_shares(secret, num_shares, prime):
    shares = [random.randint(0, prime-1) for _ in range(num_shares - 1)]
    shares.append((secret - sum(shares)) % prime)
    return shares

def reconstruct_secret(shares, prime):
    return sum(shares) % prime

def refresh_shares(shares, prime):
    new_shares = generate_shares(0, len(shares), prime)
    return [(s + n) % prime for s, n in zip(shares, new_shares)]

def secure_division(numerator, denominator, prime):
    return (numerator * pow(denominator, prime-2, prime)) % prime

# Prime number for modular arithmetic
prime = 1000000007

# Read the CSV file
df = pd.read_csv('/content/sample_data/combined_financial_data_idx.csv')

# Function to extract financial data for a company
def extract_financial_data(symbol):
    company_data = df[(df['symbol'] == symbol) & (df['2023'].notna())]
    return {
        'current_assets': int(float(company_data[company_data['account'] == 'Current Assets']['2023'].values[0])),
        'current_liabilities': int(float(company_data[company_data['account'] == 'Current Liabilities']['2023'].values[0])),
        'total_liabilities': int(float(company_data[company_data['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0])),
        'total_equity': int(float(company_data[company_data['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0])),
        'total_assets': int(float(company_data[company_data['account'] == 'Total Assets']['2023'].values[0])),
        'net_income': int(float(company_data[company_data['account'] == 'Net Income']['2023'].values[0])),
        'total_revenue': int(float(company_data[company_data['account'] == 'Operating Revenue']['2023'].values[0]))
    }

# List of companies
companies = ['AALI', 'ABMM', 'ACES', 'ACST']

# Extract data for all companies
company_data = {}
for company in companies:
    try:
        company_data[company] = extract_financial_data(company)
    except:
        print(f"Data not available for {company}")

# Number of shares (participants)
num_shares = 3

# Generate shares for financial data
shared_data = {company: {key: generate_shares(value, num_shares, prime) for key, value in data.items()}
               for company, data in company_data.items()}

def calculate_ratios(shared_data):
    current_ratio = secure_division(
        reconstruct_secret(shared_data['current_assets'], prime),
        reconstruct_secret(shared_data['current_liabilities'], prime),
        prime
    )
    debt_to_equity = secure_division(
        reconstruct_secret(shared_data['total_liabilities'], prime),
        reconstruct_secret(shared_data['total_equity'], prime),
        prime
    )
    roa = secure_division(
        reconstruct_secret(shared_data['net_income'], prime),
        reconstruct_secret(shared_data['total_assets'], prime),
        prime
    )
    profit_margin = secure_division(
        reconstruct_secret(shared_data['net_income'], prime),
        reconstruct_secret(shared_data['total_revenue'], prime),
        prime
    )
    return current_ratio, debt_to_equity, roa, profit_margin

def print_ratios(company_name, ratios):
    print(f"{company_name} Ratios:")
    print(f"Current Ratio: {ratios[0]}")
    print(f"Debt-to-Equity Ratio: {ratios[1]}")
    print(f"Return on Assets: {ratios[2]}")
    print(f"Profit Margin: {ratios[3]}")

print("Initial Ratios:")
for company in shared_data:
    initial_ratios = calculate_ratios(shared_data[company])
    print_ratios(company, initial_ratios)

# Proactive refresh and recalculation
for i in range(3):
    print(f"\nRefresh {i+1}:")
    for company in shared_data:
        for key in shared_data[company]:
            shared_data[company][key] = refresh_shares(shared_data[company][key], prime)

        refreshed_ratios = calculate_ratios(shared_data[company])
        print_ratios(company, refreshed_ratios)

# Convert ratios to real-world values
def convert_to_real_ratio(secure_ratio, prime):
    return secure_ratio / (prime / 100)

print("\nReal-world Ratios:")
for company in shared_data:
    real_ratios = [convert_to_real_ratio(ratio, prime) for ratio in calculate_ratios(shared_data[company])]
    print_ratios(company, real_ratios)

Initial Ratios:
AALI Ratios:
Current Ratio: 873546332
Debt-to-Equity Ratio: 248179188
Return on Assets: 906832761
Profit Margin: 61947636
ABMM Ratios:
Current Ratio: 727145569
Debt-to-Equity Ratio: 463225848
Return on Assets: 605220974
Profit Margin: 556689720
ACES Ratios:
Current Ratio: 94990536
Debt-to-Equity Ratio: 502829602
Return on Assets: 326009508
Profit Margin: 669807871
ACST Ratios:
Current Ratio: 398490181
Debt-to-Equity Ratio: 710337159
Return on Assets: 254834250
Profit Margin: 347984245

Refresh 1:
AALI Ratios:
Current Ratio: 873546332
Debt-to-Equity Ratio: 248179188
Return on Assets: 906832761
Profit Margin: 61947636
ABMM Ratios:
Current Ratio: 727145569
Debt-to-Equity Ratio: 463225848
Return on Assets: 605220974
Profit Margin: 556689720
ACES Ratios:
Current Ratio: 94990536
Debt-to-Equity Ratio: 502829602
Return on Assets: 326009508
Profit Margin: 669807871
ACST Ratios:
Current Ratio: 398490181
Debt-to-Equity Ratio: 710337159
Return on Assets: 254834250
Profit Margin: 34

# **NOW WITH DIFFERENT NUMBER OF SHARES AND AFFECT ON COMPUTATIONAL EFFECIENCY**

In [ ]:
import pandas as pd
import random
import time
from statistics import mean, stdev

def generate_shares(secret, num_shares, prime):
    shares = [random.randint(0, prime-1) for _ in range(num_shares - 1)]
    shares.append((secret - sum(shares)) % prime)
    return shares

def reconstruct_secret(shares, prime):
    return sum(shares) % prime

def refresh_shares(shares, prime):
    new_shares = generate_shares(0, len(shares), prime)
    return [(s + n) % prime for s, n in zip(shares, new_shares)]

def secure_division(numerator, denominator, prime):
    return (numerator * pow(denominator, prime-2, prime)) % prime

# Prime number for modular arithmetic
prime = 1000000007

# Function to extract financial data for a company
def extract_financial_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    return {
        'current_assets': int(float(company_data[company_data['account'] == 'Current Assets']['2023'].values[0])),
        'current_liabilities': int(float(company_data[company_data['account'] == 'Current Liabilities']['2023'].values[0])),
        'total_liabilities': int(float(company_data[company_data['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0])),
        'total_equity': int(float(company_data[company_data['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0])),
        'total_assets': int(float(company_data[company_data['account'] == 'Total Assets']['2023'].values[0])),
        'net_income': int(float(company_data[company_data['account'] == 'Net Income']['2023'].values[0])),
        'total_revenue': int(float(company_data[company_data['account'] == 'Operating Revenue']['2023'].values[0]))
    }

# List of companies
companies = ['AALI', 'ABMM', 'ACES', 'ACST']

# Extract data for all companies
company_data = {}
for company in companies:
    try:
        company_data[company] = extract_financial_data(company)
    except:
        print(f"Data not available for {company}")

def calculate_ratios(shared_data):
    current_ratio = secure_division(
        reconstruct_secret(shared_data['current_assets'], prime),
        reconstruct_secret(shared_data['current_liabilities'], prime),
        prime
    )
    debt_to_equity = secure_division(
        reconstruct_secret(shared_data['total_liabilities'], prime),
        reconstruct_secret(shared_data['total_equity'], prime),
        prime
    )
    roa = secure_division(
        reconstruct_secret(shared_data['net_income'], prime),
        reconstruct_secret(shared_data['total_assets'], prime),
        prime
    )
    profit_margin = secure_division(
        reconstruct_secret(shared_data['net_income'], prime),
        reconstruct_secret(shared_data['total_revenue'], prime),
        prime
    )
    return current_ratio, debt_to_equity, roa, profit_margin

def print_ratios(company_name, ratios):
    print(f"{company_name} Ratios:")
    print(f"Current Ratio: {ratios[0]}")
    print(f"Debt-to-Equity Ratio: {ratios[1]}")
    print(f"Return on Assets: {ratios[2]}")
    print(f"Profit Margin: {ratios[3]}")

def convert_to_real_ratio(secure_ratio, prime):
    return secure_ratio / (prime / 100)

def analyze_secret_sharing_efficiency(num_shares_range, num_refreshes, num_trials):
    results = []

    for num_shares in num_shares_range:
        print(f"\nAnalyzing with {num_shares} shares:")

        trial_times = []
        for _ in range(num_trials):
            start_time = time.time()

            # Generate shares for financial data
            shared_data = {company: {key: generate_shares(value, num_shares, prime) for key, value in data.items()}
                           for company, data in company_data.items()}

            # Initial calculation
            initial_ratios = {company: calculate_ratios(shared_data[company]) for company in shared_data}

            # Proactive refresh and recalculation
            for _ in range(num_refreshes):
                for company in shared_data:
                    for key in shared_data[company]:
                        shared_data[company][key] = refresh_shares(shared_data[company][key], prime)
                    calculate_ratios(shared_data[company])

            end_time = time.time()
            trial_times.append(end_time - start_time)

        avg_time = mean(trial_times)
        std_dev = stdev(trial_times)

        print(f"Average execution time: {avg_time:.4f} seconds")
        print(f"Standard deviation: {std_dev:.4f} seconds")

        results.append((num_shares, avg_time, std_dev))

    return results

# Parameters for analysis
num_shares_range = range(3, 11)  # Test with 3 to 10 shares
num_refreshes = 5
num_trials = 10

# Perform analysis
efficiency_results = analyze_secret_sharing_efficiency(num_shares_range, num_refreshes, num_trials)

# Print summary of results
print("\nEfficiency Analysis Summary:")
for num_shares, avg_time, std_dev in efficiency_results:
    print(f"Shares: {num_shares}, Avg Time: {avg_time:.4f}s, Std Dev: {std_dev:.4f}s")

# Calculate and print real-world ratios for comparison
print("\nReal-world Ratios:")
for company in company_data:
    real_ratios = [
        company_data[company]['current_assets'] / company_data[company]['current_liabilities'],
        company_data[company]['total_liabilities'] / company_data[company]['total_equity'],
        company_data[company]['net_income'] / company_data[company]['total_assets'],
        company_data[company]['net_income'] / company_data[company]['total_revenue']
    ]
    print_ratios(company, real_ratios)


Analyzing with 3 shares:
Average execution time: 0.0021 seconds
Standard deviation: 0.0002 seconds

Analyzing with 4 shares:
Average execution time: 0.0026 seconds
Standard deviation: 0.0004 seconds

Analyzing with 5 shares:
Average execution time: 0.0027 seconds
Standard deviation: 0.0001 seconds

Analyzing with 6 shares:
Average execution time: 0.0031 seconds
Standard deviation: 0.0002 seconds

Analyzing with 7 shares:
Average execution time: 0.0037 seconds
Standard deviation: 0.0005 seconds

Analyzing with 8 shares:
Average execution time: 0.0038 seconds
Standard deviation: 0.0002 seconds

Analyzing with 9 shares:
Average execution time: 0.0045 seconds
Standard deviation: 0.0012 seconds

Analyzing with 10 shares:
Average execution time: 0.0051 seconds
Standard deviation: 0.0019 seconds

Efficiency Analysis Summary:
Shares: 3, Avg Time: 0.0021s, Std Dev: 0.0002s
Shares: 4, Avg Time: 0.0026s, Std Dev: 0.0004s
Shares: 5, Avg Time: 0.0027s, Std Dev: 0.0001s
Shares: 6, Avg Time: 0.0031s

# **CHECKING USING 4 AND 7 SHARES RESPECTIVELY**

In [ ]:
import pandas as pd
import random
from decimal import Decimal, getcontext
import time

# Set high precision for decimal calculations
getcontext().prec = 30

# Prime number for modular arithmetic
prime = 1000000007

def generate_shares(secret, num_shares, prime):
    shares = [random.randint(0, prime-1) for _ in range(num_shares - 1)]
    shares.append((secret - sum(shares)) % prime)
    return shares

def reconstruct_secret(shares, prime):
    return sum(shares) % prime

def secure_division(numerator, denominator, prime):
    return (numerator * pow(denominator, prime-2, prime)) % prime

def secure_multiplication(a, b, prime):
    return (a * b) % prime

def secure_addition(a, b, prime):
    return (a + b) % prime

def secure_subtraction(a, b, prime):
    return (a - b) % prime

def convert_to_real_ratio(secure_ratio, prime):
    return secure_ratio / (prime / 100)

# Function to extract financial data for a company
def extract_financial_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    return {
        'current_assets': int(float(company_data[company_data['account'] == 'Current Assets']['2023'].values[0])),
        'current_liabilities': int(float(company_data[company_data['account'] == 'Current Liabilities']['2023'].values[0])),
        'total_liabilities': int(float(company_data[company_data['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0])),
        'total_equity': int(float(company_data[company_data['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0])),
        'total_assets': int(float(company_data[company_data['account'] == 'Total Assets']['2023'].values[0])),
        'net_income': int(float(company_data[company_data['account'] == 'Net Income']['2023'].values[0])),
        'total_revenue': int(float(company_data[company_data['account'] == 'Operating Revenue']['2023'].values[0])),
        'ebit': int(float(company_data[company_data['account'] == 'EBIT']['2023'].values[0])),
        'interest_expense': int(float(company_data[company_data['account'] == 'Interest Expense']['2023'].values[0])),
        'non_current_liabilities': int(float(company_data[company_data['account']=='Total Non Current Liabilities Net Minority Interest']['2023'].values[0]))
    }

# List of companies
companies = ['AALI', 'ABMM', 'ACES', 'ACST']

# Extract data for all companies
company_data = {}
for company in companies:
    try:
        company_data[company] = extract_financial_data(company)
    except:
        print(f"Data not available for {company}")

def generate_and_reconstruct(secret, num_shares, prime):
    # Generate shares
    shares = generate_shares(secret, num_shares, prime)

    # Reconstruct secret
    reconstructed = reconstruct_secret(shares, prime)

    return reconstructed

def calculate_ratios_with_shares(company_data, num_shares, prime):
    reconstructed_data = {}
    for key, value in company_data.items():
        reconstructed_data[key] = generate_and_reconstruct(value, num_shares, prime)

    ratios = {}
    times = {}

    start_time = time.time()
    ratios['current_ratio'] = secure_division(reconstructed_data['current_assets'], reconstructed_data['current_liabilities'], prime)
    times['current_ratio'] = time.time() - start_time

    start_time = time.time()
    ratios['debt_to_equity'] = secure_division(reconstructed_data['total_liabilities'], reconstructed_data['total_equity'], prime)
    times['debt_to_equity'] = time.time() - start_time

    start_time = time.time()
    ratios['roa'] = secure_division(reconstructed_data['net_income'], reconstructed_data['total_assets'], prime)
    times['roa'] = time.time() - start_time

    start_time = time.time()
    ratios['profit_margin'] = secure_division(reconstructed_data['net_income'], reconstructed_data['total_revenue'], prime)
    times['profit_margin'] = time.time() - start_time

    start_time = time.time()
    ratios['asset_turnover'] = secure_division(reconstructed_data['total_revenue'], reconstructed_data['total_assets'], prime)
    times['asset_turnover'] = time.time() - start_time

    start_time = time.time()
    ebit_plus_interest = secure_addition(reconstructed_data['ebit'], reconstructed_data['interest_expense'], prime)
    ratios['interest_coverage'] = secure_division(ebit_plus_interest, reconstructed_data['interest_expense'], prime)
    times['interest_coverage'] = time.time() - start_time

    start_time = time.time()
    total_debt = secure_addition(reconstructed_data['current_liabilities'], reconstructed_data['non_current_liabilities'], prime)
    ratios['debt_ratio'] = secure_division(total_debt, reconstructed_data['total_assets'], prime)
    times['debt_ratio'] = time.time() - start_time

    return ratios, times

# Calculate ratios for 4 and 7 shares
for num_shares in [4, 7]:
    print(f"\nReconstructing ratios with {num_shares} shares:")
    for company, data in company_data.items():
        ratios, times = calculate_ratios_with_shares(data, num_shares, prime)
        print(f"\n{company} Ratios:")
        for ratio_name, ratio_value in ratios.items():
            print(f"{ratio_name}: {convert_to_real_ratio(ratio_value, prime):.4f} (Time: {times[ratio_name]:.6f} seconds)")

# Print real-world ratios for comparison
print("\nReal-world Ratios:")
for company, data in company_data.items():
    real_ratios = {
        'current_ratio': data['current_assets'] / data['current_liabilities'],
        'debt_to_equity': data['total_liabilities'] / data['total_equity'],
        'roa': data['net_income'] / data['total_assets'],
        'profit_margin': data['net_income'] / data['total_revenue'],
        'asset_turnover': data['total_revenue'] / data['total_assets'],
        'interest_coverage': (data['ebit'] + data['interest_expense']) / data['interest_expense'],
        'debt_ratio': (data['current_liabilities'] + data['non_current_liabilities']) / data['total_assets']
    }
    print(f"\n{company} Ratios:")
    for ratio_name, ratio_value in real_ratios.items():
        print(f"{ratio_name}: {ratio_value:.4f}")



Reconstructing ratios with 4 shares:

AALI Ratios:
current_ratio: 87.3546 (Time: 0.000011 seconds)
debt_to_equity: 24.8179 (Time: 0.000007 seconds)
roa: 90.6833 (Time: 0.000007 seconds)
profit_margin: 6.1948 (Time: 0.000007 seconds)
asset_turnover: 11.8058 (Time: 0.000007 seconds)
interest_coverage: 89.9973 (Time: 0.000009 seconds)
debt_ratio: 26.1232 (Time: 0.000008 seconds)

ABMM Ratios:
current_ratio: 72.7146 (Time: 0.000011 seconds)
debt_to_equity: 46.3226 (Time: 0.000008 seconds)
roa: 60.5221 (Time: 0.000007 seconds)
profit_margin: 55.6690 (Time: 0.000007 seconds)
asset_turnover: 71.7848 (Time: 0.000007 seconds)
interest_coverage: 21.5923 (Time: 0.000009 seconds)
debt_ratio: 23.2191 (Time: 0.000008 seconds)

ACES Ratios:
current_ratio: 9.4991 (Time: 0.000008 seconds)
debt_to_equity: 50.2830 (Time: 0.000006 seconds)
roa: 32.6010 (Time: 0.000007 seconds)
profit_margin: 66.9808 (Time: 0.000006 seconds)
asset_turnover: 93.0184 (Time: 0.000007 seconds)
interest_coverage: 20.7590 (Time